## Notes for next week:
* filter down json file
  * flatten the data
  * remove the retweets
  * save in format usable by nltk or scikit learn tools
* Scikit for text analysis 
* Answering the questions for exploration of data
* Decide an actual question to answer
  * Media and text in combination


In [2]:
import pandas
import json
import ijson
import pprint
from itertools import islice

In [3]:
# set file path for tweets dataset
file_path = '/Users/adamstueckrath/Desktop/SyriaProjectNotes/data/tweets/tweets.json'

# read n number of json objects from the tweets dataset
def read_n_from_file(file_name, n_lines):
    data = []
    with open(file_name) as f:
        for line in islice(f, n_lines):
            data.append(json.loads(line))
        return(data)  

# get sample json objects 
sample_json_objects = read_n_from_file(file_path, 2)

# print sample
pprint.pprint(sample_json_objects)


[{'_id': {'$oid': '595e82d713bbf01307babbba'},
  'contributors': None,
  'coordinates': None,
  'created_at': {'$date': '2017-07-06T18:34:37.000Z'},
  'entities': {'hashtags': [{'indices': [79, 88], 'text': 'Basirhat'}],
               'symbols': [],
               'urls': [{'expanded_url': None,
                         'indices': [134, 134],
                         'url': ''}],
               'user_mentions': [{'id': 147994804,
                                  'id_str': '147994804',
                                  'indices': [3, 15],
                                  'name': 'Rishi Bagree 🇮🇳',
                                  'screen_name': 'rishibagree'},
                                 {'id': 37034483,
                                  'id_str': '37034483',
                                  'indices': [117, 122],
                                  'name': 'NDTV',
                                  'screen_name': 'ndtv'}]},
  'favorite_count': 0,
  'favorited': False,
  'filter_

In [7]:
!pip install line_profiler
%load_ext line_profiler

In [9]:
# %%timeit
# get the total number of json objects in file 
# json objects must be stored per line, not in an arrary 
# file contains separate JSON object on each line.


def count_json_objects(file_name):
    count = 0
    with open(file_name, 'r') as file:
        for line in file: 
            count+=1
        return count
    
# print total number of json objects

count_json_objects(file_path)



/Users/adamstueckrath/.pyenv/versions/3.6.5/lib/python3.6/site-packages/line_profiler.py:321: UserWarning: Could not extract a code object for the object 2853841
  profile = LineProfiler(*funcs)


In [11]:
%lprun -f count_json_objects count_json_objects(file_path)

In [25]:

start = time.time()
print("hello")

def count_json_objects_v2(file_name):
    count = 0
    with open(file_path) as f:
        while True:
            next_n_lines = list(islice(f, 20))
            if not next_n_lines:
                break
            for line in next_n_lines: 
                count += 1
        return count

json_objects_total = count_json_objects_v2(file_path)
print(json_objects_total)
    
end = time.time()
print(end - start)



hello
2853841
39.1446270942688


In [146]:
# get column names and types
def get_columns_types(dataframe):
    column_details = {}
    columns = dataframe.columns.values.tolist()
    for column in columns: 
        column_details[column] = type(dataframe[column].iat[0])
    return column_details


In [147]:
from pandas.io.json import json_normalize

# new line json streamer
def nljson_generator(file_name):
    with open(file_name) as file:
        for line in file: 
            yield json.loads(line)
        f.close()
        
# load json to dataframe
def json_to_dataframe(json_object):
    dataframe = json_normalize(json_object)
    return dataframe

# source code: https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10
# flattens nested json
def flatten_json(json_object):
    out = {}
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x
    flatten(json_object)
    return out


In [174]:
from collections import Counter

# not using this function 
def get_counts_of_values(dictionary, key):
    c = Counter()
    for item in dictionary:
        if key in item:
            c[item[key]] += 1
    return c



def run():
    
    json_stream = nljson_generator(file_path)
    tweet_limit = 0
    c = Counter()
    
    for tweet in json_stream:
        tweet_limit += 1        
        flatten_tweet = flatten_json(tweet)
        
        if tweet_limit == 1000000:
            return c 
            break
        
        for col, item in flatten_tweet.items():
            if col == 'text' and "Syria" in item:
                c['Syria'] += 1

start = time.time()
print("hello")
a = run()
print(a)
end = time.time()
print(end - start)

hello
Counter({'Syria': 579466})
248.3317940235138
